In [1]:
from tinybig.config import config
from tinybig.util import set_random_seed

print('loading configs...')
config_file_name = 'diabetes_configs'
config_obj = config()
config = config_obj.load_yaml(cache_dir='./configs', config_file=config_file_name + '.yaml')
print(config)

loading configs...
{'configurations': {'device': 'mps', 'random_seed': 1234, 'data_configs': {'data_class': 'tinybig.data.diabetes', 'data_parameters': {'name': 'diabetes', 'train_batch_size': 1000, 'test_batch_size': 1000}}, 'learner_configs': {'learner_class': 'tinybig.learner.backward_learner', 'learner_parameters': {'name': 'error_backward_propagation', 'n_epochs': 3000, 'optimizer_configs': {'optimizer_class': 'torch.optim.AdamW', 'optimizer_parameters': {'lr': 0.005, 'weight_decay': 1e-05}}, 'loss_configs': {'loss_class': 'torch.nn.MSELoss', 'loss_parameters': {'reduction': 'mean'}}}}, 'model_configs': {'model_class': 'tinybig.model.rpn', 'model_parameters': {'device': 'mps', 'name': 'reconciled_polynomial_network', 'depth': 1, 'depth_alloc': [1], 'layer_configs': [{'layer_class': 'tinybig.module.rpn_layer', 'layer_parameters': {'name': 'rpn_layer', 'm': 8, 'n': 1, 'width': 1, 'width_alloc': [1], 'head_configs': [{'head_class': 'tinybig.module.rpn_head', 'head_parameters': {'l': 

In [2]:
print('setting up environments...')
device = config['configurations'].pop('device')
random_seed = config['configurations'].pop('random_seed')
set_random_seed(random_seed)
print('device: ', device, '; random_seed: ', random_seed)

setting up environments...
device:  mps ; random_seed:  1234


In [3]:
print('instantiating objects from config...')
data_obj, model_obj, learner_obj, metric_obj, result_obj = [config_obj.instantiation_from_configs(config['configurations'][f'{stem}_configs'], device=device, class_name=f'{stem}_class', parameter_name=f'{stem}_parameters') for stem in ['data', 'model', 'learner', 'metric', 'output']]

print('parameter num: ', sum([parameter.numel() for parameter in model_obj.parameters()]))


instantiating objects from config...
combinatorial_normal_expansion initialization
parameter num:  44


In [4]:
print('loading dataset...')
data_loader = data_obj.load(train_percentage=0.9, normalize_X=True)

loading dataset...


In [5]:
print('training model...')
training_records = learner_obj.train(model=model_obj, data_loader=data_loader, device=device, metric=metric_obj, disable_tqdm=True, display_step=100)
model_obj.save_ckpt(cache_dir='./ckpt', checkpoint_file=f'{config_file_name}_checkpoint')

training model...
Epoch: 0, Test Loss: 3.7307851314544678, Test Score: 3.730784991735201, Time Cost: 0.08037281036376953
Epoch: 100, Test Loss: 0.21705354750156403, Test Score: 0.2170535601839862, Time Cost: 0.004579782485961914
Epoch: 200, Test Loss: 0.20724423229694366, Test Score: 0.20724423370726192, Time Cost: 0.0040740966796875
Epoch: 300, Test Loss: 0.19777674973011017, Test Score: 0.19777675290644847, Time Cost: 0.004434823989868164
Epoch: 400, Test Loss: 0.18946895003318787, Test Score: 0.1894689608883343, Time Cost: 0.004333019256591797
Epoch: 500, Test Loss: 0.18237577378749847, Test Score: 0.18237576561451507, Time Cost: 0.004703998565673828
Epoch: 600, Test Loss: 0.17636185884475708, Test Score: 0.1763618507896189, Time Cost: 0.004437923431396484
Epoch: 700, Test Loss: 0.17128917574882507, Test Score: 0.17128916505898378, Time Cost: 0.0048906803131103516
Epoch: 800, Test Loss: 0.16704346239566803, Test Score: 0.1670434646723922, Time Cost: 0.004873991012573242
Epoch: 900, 

In [6]:
print('testing model...')
test_result = learner_obj.test(model=model_obj, test_loader=data_loader['test_loader'], device=device,
                               metric=metric_obj)

testing model...


In [7]:
print('evaluating result...')
print(metric_obj.__class__.__name__, metric_obj.evaluate(y_true=test_result['y_true'], y_pred=test_result['y_pred'], y_score=test_result['y_score'], ))

evaluating result...
mse 0.14715478886237063


In [8]:
from tinybig.metric import accuracy

y_rounded_label = [[round(y[0])] for y in test_result['y_score']]
test_result['y_pred'] = y_rounded_label

acc_metric = accuracy('accuracy_metric')
print('evaluating rounded prediction labels...')
print(acc_metric.__class__.__name__, acc_metric.evaluate(y_true=test_result['y_true'], y_pred=test_result['y_pred'], y_score=test_result['y_score'], ))

evaluating rounded prediction labels...
accuracy 0.8181818181818182


In [9]:
print('saving result...')
result_obj.save(test_result, cache_dir='./result', output_file='{}_result'.format(config_file_name))
result_obj.save(training_records, cache_dir='./result', output_file='{}_record'.format(config_file_name))

saving result...
